In [1]:
import numpy as np
from pyscf import gto, scf, fci
from tabulate import tabulate
from RDMFS import HF_U2RDM, MU_U2RDM, GU_U2RDM, BBC1_U2RDM
from utils import spectral_clean, DM2_DiagSum
import scipy 

In [2]:
# Get Hydrogen in mo basis from PySCF

mol = gto.Mole()
mol.unit = 'B' 
L = 10.5
mol.atom = f"""
# He 0 0 0 
H   {-L}   {-L}   0.0
H    {L}   {-L}   0.0
H    {L}    {L}   0.0
H   {-L}    {L}   0.0
"""
# this basis has 2 functions for Helium
# mol.basis = "ccpvdz"
mol.basis = "sto-3g"
# mol.basis = "631g"

mol.spin =  0
mol.verbose= 0
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri_ao = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

## Run Hartree-Fock.
mf = scf.UHF(mol)
mf.kernel()


# Harvest HF quantities

h1 = mf.get_hcore()
M = h1.shape[0]
C_a = mf.mo_coeff[0,:,:]
C_b = mf.mo_coeff[1,:,:]

h1_a = C_a.T@h1@C_a
h1_b = C_b.T@h1@C_b

eri_h2_aa = eri_ao.copy()
eri_h2_bb = eri_ao.copy()
eri_h2_ab = eri_ao.copy()


for i in range(4):
  eri_h2_aa = np.tensordot(eri_h2_aa, C_a, axes=1).transpose(3, 0, 1, 2)
  eri_h2_bb = np.tensordot(eri_h2_bb, C_b, axes=1).transpose(3, 0, 1, 2)

eri_h2_ab = np.tensordot(eri_h2_ab, C_a, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_a, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_b, axes=1).transpose(3, 0, 1, 2)
eri_h2_ab = np.tensordot(eri_h2_ab, C_b, axes=1).transpose(3, 0, 1, 2)

h1_mo = (h1_a, h1_b)
h2_mo = (eri_h2_aa, eri_h2_ab, eri_h2_bb)

In [3]:
dm1_mo = mf.make_rdm1()
occ_mo_a, occ_mo_b, C_mo_a, C_mo_b = spectral_clean(dm1_mo, h1.shape[0])
print(occ_mo_a, occ_mo_b)


[1. 1. 0. 0.] [1.00000000e+00 1.00000000e+00 1.52436579e-16 0.00000000e+00]


In [4]:
dm2_HF_mo = HF_U2RDM(occ_mo_a, occ_mo_b,M)
dm2_MU_mo = MU_U2RDM(occ_mo_a, occ_mo_b,M)
dm2_GU_mo = GU_U2RDM(occ_mo_a, occ_mo_b,M)
dm2_BBC1_mo = BBC1_U2RDM(occ_mo_a, occ_mo_b,M,mol.nelec[0],mol.nelec[1])


In [5]:
# # Run FCI  
# cisolver = fci.direct_uhf.FCI()
# cisolver.max_cycle = 100
# cisolver.conv_tol = 1e-8
# e_mo, ci_mo = cisolver.kernel(h1_mo, h2_mo, h1.shape[0], mol.nelec, ecore=mf.energy_nuc())
# dm1_mo, dm2_mo  = cisolver.make_rdm12s(ci_mo, h1.shape[0], mol.nelec)

In [6]:
print(DM2_DiagSum(dm2_BBC1_mo), scipy.special.binom(4,2))

12.000000000000004 6.0
